# Autoencoder

### Import main libraries

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.decomposition import PCA

from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist, mnist
from tensorflow.keras.models import Model


Autoencoders are a data compression algorithms made up of a compression and decompression functions.
Two interesting practical applications of autoencoders are data denoising, and dimensionality reduction for data visualization. 

 The lab is organized as following:

  1.1 Dataset loading;

  1.2 Pre-processing (Dataset normalization, splitting and label pre-processing; 

  1.3 Deep Autoencoder implementation;

  1.4 Autoencoder training;

  1.5 Autoencoder testing.


  Similarly with a Convolutional Autoencoder.

  You will visualize the latent representation and use such method for image denoising with synthetic noise.




 

### Load data
Load the data you want experiment with (e.g. fashion and mnist). Experiment with different dataset, how the architectures should be changed? Do you notice any difference in the results (e.g. in terms of the loss function)?

In [ ]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

print(x_train.shape)
print(x_test.shape)

Normalize and add one channel.
One common practice in training a Neural Network is to normalize the images by dividing each pixel value by the maximum value that we can have.<br> The purpose of this is to obtain a mean close to 0.<br>
Normalizing the data generally speeds up learning and leads to faster convergence

In [ ]:
def preprocess(array):
    """
    Normalizes the supplied array and reshapes it into the appropriate format.
    """
    array = array.astype("float32") / # Fill here#
    
    # add 1 channel
    array = np.reshape(array, (len(array), # Fill here #))
    return array

# Normalize and reshape the data
x_train = preprocess(x_train)
x_test = preprocess(x_test)



## Visualize data

In [ ]:
def plot_imgs(imgs, n= 20):
  plt.figure(figsize=(20, 4))
  for i in range(n):
    # Display original
    ax = plt.subplot(1, n, i + 1)
    plt.imshow(imgs[i])
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

plot_imgs(x_train)

### Define Deep Autoencoder with Dense layers

Models in Keras are defined as a sequence of layers. The things to choose when defining the architecture are many:
 - number of layers
 - type of layers
 - size of layers
 - type of non-linearity (activation functions)
 - whether or not to add regularization
 
Here we will use only fully-connected (dense) layers, so the type of layer is fixed. Fully connected layers are defined using the [Dense](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense) class, which takes as parameters the number of neurons (which is the **dimension of the output**).

The following image is sketch of the general architecure:

![Deep Autoencoder](https://www.compthree.com/images/blog/ae/ae.png)


Define a class extending [Model](https://www.tensorflow.org/api_docs/python/tf/keras/Model), then we define the [sequence](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential) of layers for both encoder and decoder. Note than the Encoder and Decoder  should be as symmetric as possible. 

In [ ]:


class Autoencoder(Model):
  def __init__(self, latent_dim):
    super(Autoencoder, self).__init__()
    self.latent_dim = latent_dim   


    self.encoder = tf.keras.Sequential([
      layers.Flatten(),
      layers.Dense(128, activation='relu'),
      layers.Dense(64, activation='relu'),
      layers.Dense(latent_dim, activation='relu'),
    ])

    
    self.decoder = tf.keras.Sequential([
      layers.Dense(64, activation='relu'),
      layers.Dense(128, activation='relu'),
      layers.Dense(784, activation='sigmoid'),
      layers.Reshape((28, 28))
    ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded



## Compile model

Define the autoencoder of fully connected layers.
After having created a model you need to **compile** it. During the compilation phase you must specify some parameters related to how the model will be optimized:
 - The `optimizer`. For the following exercise you should use [SGD](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/SGD), initialized with some learning rate.
 - The `loss` function. For the reconstruction you can use the [mean squared error](https://www.tensorflow.org/api_docs/python/tf/keras/losses/MeanSquaredError) loss.
 - A list of `metrics`: common error functions which you want keras to report at each training epoch.



In [ ]:
latent_dim = 10
autoencoder = Autoencoder( latent_dim)

#  set optimizer and loss function
autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())


In [ ]:

epochs = 5
history = autoencoder.fit(x_train, x_train,
                epochs=epochs,
                shuffle=True,
                validation_data=(x_test, x_test))


autoencoder.summary()

Plot the loss function for each epoch for both validation and traning data. See [History](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/History).

In [ ]:
def plot_loss(history):
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  plt.plot(loss, marker="o", c="red", label='Training loss')
  plt.plot(val_loss, 'b', label='Validation loss')
  plt.title('Training and validation loss')
  plt.xlabel("epochs")
  plt.ylabel("loss")
  plt.legend()

  plt.show()

plot_loss(history)

Test the learnt model on the test set.

In [ ]:
encoded_imgs = autoencoder.encoder(x_test).numpy()
decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()



In [ ]:
def plot_examples(x_test, decoded_imgs, n = 10):
  
  plt.figure(figsize=(20, 4))
  for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i])
    plt.title("original")
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i])
    plt.title("reconstructed")
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
  plt.show()

plot_examples(x_test,decoded_imgs )

Visualize the latent representation. If it is 2-dimensional plot as it is, otherwise apply PCA dimensionality reduction. Try to visualize the points with a different color for each class, what do you observe?

In [ ]:
def plot_pca_latent_space(encoded_imgs, labels):

  pca = PCA(n_components=2)
  data_reduced = pca.fit_transform(encoded_imgs)

  x, y = data_reduced[..., 0], data_reduced[..., 1]

  plt.figure(figsize=(8, 6), dpi=80)
  plt.scatter(x, y, c=labels, cmap='viridis')
  plt.title("Representation wrt labels")
  plt.colorbar()
  plt.show()

def plot_latent_space(encoded_imgs, labels):

  assert encoded_imgs.shape[-1] ==2

  x, y = encoded_imgs[..., 0], encoded_imgs[..., 1]

  plt.figure(figsize=(8, 6), dpi=80)
  plt.scatter(x, y, c=labels, cmap='viridis')
  plt.title("Representation wrt labels")
  plt.colorbar()
  plt.show()

if latent_dim==2:
  plot_latent_space(encoded_imgs, y_test)
else:
  plot_pca_latent_space(encoded_imgs, y_test)

Here we will scan the latent plane, sampling latent points at regular intervals, and generating the corresponding image for each of these points. This gives us a visualization of the latent manifold that "generates" the data.

In [ ]:
def plot_manifold(decoder, n=15):
  # Display a 2D manifold of the digits
  n = 15  # figure with 15x15 digits
  img_size = 28
  figure = np.zeros((img_size * n, img_size * n))
  # We will sample n points within [-15, 15] standard deviations
  grid_x = np.linspace(-15, 15, n)
  grid_y = np.linspace(-15, 15, n)

  for i, yi in enumerate(grid_x):
      for j, xi in enumerate(grid_y):
          z_sample = np.array([[xi, yi]])
          print(z_sample.shape)
          x_decoded = decoder.predict(z_sample)
          digit = x_decoded[0].reshape(img_size, img_size)
          figure[i * img_size: (i + 1) * img_size,
               j * img_size: (j + 1) * img_size] = digit

  plt.figure(figsize=(10, 10))
  plt.imshow(figure)
  plt.show()

if latent_dim==2:
  plot_manifold(autoencoder.decoder)

Experiment with different latent dimensions and comment the results you obtain.
You may try different architectures. 
We ask you to add one layer, again,discuss the results.

### Define Autoencoder with Convolutional layers

we are going to create a convolutional model in Keras. 
Usually a convolutional model is made by two subsequent part:
* A convolutional part
* A fully connected

We can show an example of the general structure in the next picture:

![Convolutional autoencoder](https://149695847.v2.pressablecdn.com/wp-content/uploads/2020/07/The-structure-of-proposed-Convolutional-AutoEncoders-CAE-for-MNIST-In-the-middle-there.png)



Usually the convolutional part is made by some layers composed by
* convolutional layer: performs a spatial convolution over images
* pooling layer: used to reduce the output spatial dimension from $n$ to 1 by averaging the $n$ different value or considering the maximum between them 
* dropout layer: applied to a layer, consists of randomly "dropping out" (i.e. set to zero) a number of output features of the layer during training.



Implement a Model as you have done for the deep autoencoder. Here try to use Convolutions, Batch Normalizations and Pooling layers.

The Decoder should be specular to the Encoder, therefore the Convolutional layers should be replaced with [Conv2DTranspose](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2DTranspose)

In [ ]:

class ConvAutoencoder(Model):
  def __init__(self, latent_dim):
    super(ConvAutoencoder, self).__init__()
    self.latent_dim = latent_dim   
    self.encoder = tf.keras.Sequential([
      # Fill here #
    ])
    self.decoder = tf.keras.Sequential([
     # Fill here #
    ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

In [ ]:
latent_dim = 64 
autoencoder = ConvAutoencoder(latent_dim)

#  set optimizer and loss function
autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

epochs = 10
history = autoencoder.fit(x_train, x_train,
                epochs=epochs,
                shuffle=True,
                validation_data=(x_test, x_test))

encoded_imgs = autoencoder.encoder(x_test).numpy()
decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()

autoencoder.summary()

In [ ]:
plot_loss(history)

In [ ]:
plot_examples(x_test,decoded_imgs )



In [ ]:
if latent_dim==2:
  plot_latent_space(encoded_imgs, y_test)
else:
  plot_pca_latent_space(encoded_imgs, y_test)

Do you see any difference between the Deep Autoencoder and the Convolutional one? Training time, Reconstruction loss, Complexity of the model (e.g. number of parameters).

## Denoising (optional)

Let's put our convolutional autoencoder to work on an image denoising problem. It's simple: we will train the autoencoder to map noisy images to clean images.

Here's how we will generate synthetic noisy images: we just apply a gaussian noise matrix and clip the images between 0 and 1.



In [ ]:
def add_noise(img, noise_factor = 0.5):
  # add gaussian noise
  img_noisy = img + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=img.shape) 
  img_noisy = np.clip(img_noisy, 0., 1.)
  return img_noisy

x_train_noisy = add_noise(x_train)
x_test_noisy = add_noise(x_test)

In [ ]:
latent_dim = 64 
autoencoder = ConvAutoencoder(latent_dim)

#  set optimizer and loss function
autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

epochs = 10
autoencoder.fit(# fill here #
                , # fill here# ,
                epochs=epochs,
                shuffle=True,
                validation_data=(
                    #fill here#
                ))